In [ ]:
#import libraries
from bs4 import BeautifulSoup as bs
import requests 
import time
import datetime
import pandas as pd
import pprint

import smtplib

In [ ]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")

#convert to beautiful soup object
soup = bs(page.content)

#print out the HTML
contents = soup.prettify()
print(contents)

In [ ]:
#find all elements in the page
td_elements = soup.find_all('td')
print(td_elements)

In [ ]:
#Just checking
# iterate over each <td> element and extract the title
for td in td_elements:
    title_element = td.find('a')
    if title_element:
        title = title_element.get('title')
        print(title)

In [ ]:
# iterate over each <td> element
for td in td_elements:
    # find the first <a> element within the <td> element
    link_element = td.find('a')

    # check if <a> element exists
    if link_element:
        # get the title and href attributes
        title = link_element.get('title')
        href = link_element.get('href')

        # print the title and link
        print("Title:", title)
        print("Link:", href)

In [ ]:
# Function to extract actual content value
def actual_content_value(row_data):
    # Remove all sup tags
    for sup in row_data.find_all("sup"):
        sup.decompose()
    # Check if row has a list <li> in the HTML
    if row_data.find("li"):
        return [li.get_text(strip=True).replace('\n', ' ') for li in row_data.find_all("li")]
    #check for <br> and split accordingly
    elif row_data.find("br"):
        return [text for text in row_data.stripped_strings]
    else:
        return row_data.find("td").get_text(strip=True).replace('\n', '').replace('\xa0', ' ')
def get_info_box(url):
    page = requests.get(url)

    # convert to beautiful soup object
    soup = bs(page.content)

    # extract all the rows
    info_box = soup.find(class_="infobox vevent")
    info_rows = info_box.find_all("tr")

    # Create an empty dictionary to store the data
    movie_info = {}

    for index, row in enumerate(info_rows):
        if index == 0:
            movie_info['title'] = row.find("th").get_text().strip()
        elif index == 1:
            continue
        else:
            content_key = row.find("th").get_text(" ", strip=True).replace('\n', ' ')
            content_value = actual_content_value(row)
            movie_info[content_key] = content_value

            
                # Replace '\xa0' with a space in the release date value
        release_date_key = 'Release date' if 'Release date' in movie_info else 'Release dates'
        modified_release_date = movie_info.get(release_date_key, [''])[0].replace('\xa0', ' ')

        # Update the release date value in the dictionary
        movie_info[release_date_key] = [modified_release_date]

        # Check if the key is 'Production company' or 'Production companies'
        production_key = 'Production company' if 'Production company' in movie_info else 'Production companies'

        # Remove the space between the items in 'Production companies' list
        production_value = movie_info.get(production_key, [])
        if isinstance(production_value, list):
            production_companies = [company.replace(' ', '') for company in production_value]
            movie_info[production_key] = production_companies
    pprint.pprint(movie_info)
    return movie_info

In [ ]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")
soup = bs(page.content)
all_td_elements = soup.find_all('td')

base_path = "https://www.wikipedia.org/"

movie_info_list = []

for td in all_td_elements:
    link_element = td.find('a')

    if link_element:
        title = link_element.get('title')
        href = link_element.get('href')

        if title and title != 'None' and title != '20th Digital Studio' and not title.startswith('List') and not title.startswith('Walt'):
            print("Title:", title) 
            print("Link:", href)
            try:
                relative_path = href
                full_path = base_path + relative_path

                movie_info_list.append(get_info_box(full_path))

            except Exception as e:
                print(href)
                print(e)

In [ ]:
len(movie_info_list)

In [ ]:
save_path = r"G:\my_projects\disney\disney_data_cleaned.json"
load_path = r"G:\my_projects\disney\disney_data_cleaned.json"

In [ ]:
#save and reload data
import json

def save_data(title, data):
    with open(title, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
# Saving data to the specified location
save_path = r"G:\my_projects\disney\disney_data_cleaned.json"
save_data(save_path, movie_info_list)
    

In [1]:
import json

def load_data(title):
    with open(title, 'r', encoding='utf-8') as f:
        return json.load(f)

# Loading data from the specified location
load_path = r"G:\my_projects\disney\disney_data_cleaned.json"
movie_info_list = load_data(load_path)


In [2]:
movie_info_list[-5]

{'title': 'Pirates of the Caribbean',
 'Release dates': [''],
 'Production companies': ['Production',
  'companies',
  'WaltDisneyPictures',
  'JerryBruckheimerFilms'],
 'Directed by': ['Directed by',
  'Gore Verbinski',
  '(1–3)',
  'Rob Marshall',
  '(4)',
  'Espen Sandberg',
  '(5)',
  'Joachim Rønning',
  '(5)'],
 'Screenplay by': ['Screenplay by',
  'Ted Elliott',
  '(1–4)',
  'Terry Rossio',
  '(1–4)',
  'Jeff Nathanson',
  '(5)'],
 'Story by': ['Story by',
  'Ted Elliott',
  '(1–4)',
  'Terry Rossio',
  '(1–5)',
  'Stuart Beattie',
  '(1)',
  'Jay Wolpert',
  '(1)',
  'Jeff Nathanson',
  '(5)'],
 'Based on': ['Based on',
  'Pirates of the Caribbean',
  'by',
  'Walt Disney',
  'On Stranger Tides',
  'by',
  'Tim Powers',
  '(4)'],
 'Produced by': 'Jerry Bruckheimer',
 'Starring': ['Starring',
  'Johnny Depp',
  '(1–5)',
  'Geoffrey Rush',
  '(1–5)',
  'Kevin McNally',
  '(1–5)',
  'Orlando Bloom',
  '(1–3, 5)',
  'Keira Knightley',
  '(1–3, 5)',
  '(',
  'See below',
  ')'],
 'M

In [3]:
for movie_info in movie_info_list:
    if 'Running time' in movie_info:
        running_time_data = movie_info['Running time']
        running_time = None  # Initialize running_time as None

        # Check the type of 'Running time' value
        if isinstance(running_time_data, list):
            for item in running_time_data:
                if isinstance(item, int):
                    running_time = item
                    break  # Stop searching once an integer item is found
                elif isinstance(item, str):
                    # Try to extract numeric value from the string
                    try:
                        running_time = int(item.split()[0])
                        break  # Stop searching once a valid string item is found
                    except ValueError:
                        pass  # Handle cases where the string cannot be converted to an integer
        elif isinstance(running_time_data, (int, str)):
            # If 'Running time' is already an integer or string, try to convert it to an integer
            try:
                running_time = int(str(running_time_data).split()[0])
            except ValueError:
                pass  # Handle cases where the string cannot be converted to an integer

        # Update 'Running time' with the extracted value or None if not found
        movie_info['Running time'] = running_time

# Now, the 'Running time' values in movie_info_list have been updated to integers or None if not found
for movie_info in movie_info_list:
    print(movie_info.get('Running time'))

83
88
126
74
64
70
42
65
71
75
94
73
75
82
68
74
96
75
84
77
92
69
81
60
127
93
76
75
73
85
81
70
90
80
75
84
83
72
97
75
104
93
105
95
97
134
69
92
126
79
97
128
73
91
105
98
130
89
93
67
98
100
118
103
110
80
79
91
91
97
118
139
None
131
92
87
116
93
114
110
131
101
110
84
78
75
164
106
110
99
113
108
102
85
91
93
100
100
79
96
113
89
118
92
88
92
87
93
93
93
90
83
96
88
89
91
93
92
97
100
100
89
None
91
112
115
95
91
97
104
74
77
104
128
101
94
104
90
100
88
93
98
112
84
97
97
114
96
97
109
83
90
107
96
103
91
95
105
113
80
101
90
74
90
89
110
74
93
84
83
69
77
107
93
88
108
84
121
89
104
90
86
84
108
107
96
98
105
109
94
106
102
69
88
102
102
97
111
92
100
96
96
78
81
108
89
99
89
81
92
100
89
79
91
81
101
104
103
86
106
74
93
92
98
76
95
72
93
87
70
93
87
128
72
77
82
86
95
114
93
83
83
88
78
70
112
66
92
75
79
79
72
82
104
70
75
113
100
78
83
69
96
115
86
92
65
95
73
73
68
128
86
75
88
68
96
104
95
74
72
88
75
61
117
94
80
100
143
96
64
87
85
87
50
74
136
78
85
65
76
40
120
84
11

In [4]:
movie_info_list[-9]

{'title': 'The Rocketeer',
 'Release dates': [''],
 'Production companies': ['WaltDisneyPictures',
  'TouchstonePictures',
  'SilverScreenPartnersIV',
  'GordonCompany'],
 'Directed by': 'Joe Johnston',
 'Screenplay by': ['Danny Bilson', 'Paul De Meo'],
 'Story by': ['Danny Bilson', 'Paul De Meo', 'William Dear'],
 'Based on': ['Based on', 'The Rocketeer', 'by', 'Dave Stevens'],
 'Produced by': ['Charles Gordon', 'Lawrence Gordon', 'Lloyd Levin'],
 'Starring': ['Billy Campbell',
  'Alan Arkin',
  'Jennifer Connelly',
  'Paul Sorvino',
  'Timothy Dalton'],
 'Cinematography': 'Hiro Narita',
 'Edited by': ['Michael A. Stevenson', 'Arthur Schmidt'],
 'Music by': 'James Horner',
 'Distributed by': 'Buena Vista Pictures Distribution',
 'Release date': ['June 21, 1991(1991-06-21)'],
 'Running time': 108,
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$35–40 million',
 'Box office': '$46.7 million (U.S.)'}

In [5]:
movie_info_list[-5]

{'title': 'Pirates of the Caribbean',
 'Release dates': [''],
 'Production companies': ['Production',
  'companies',
  'WaltDisneyPictures',
  'JerryBruckheimerFilms'],
 'Directed by': ['Directed by',
  'Gore Verbinski',
  '(1–3)',
  'Rob Marshall',
  '(4)',
  'Espen Sandberg',
  '(5)',
  'Joachim Rønning',
  '(5)'],
 'Screenplay by': ['Screenplay by',
  'Ted Elliott',
  '(1–4)',
  'Terry Rossio',
  '(1–4)',
  'Jeff Nathanson',
  '(5)'],
 'Story by': ['Story by',
  'Ted Elliott',
  '(1–4)',
  'Terry Rossio',
  '(1–5)',
  'Stuart Beattie',
  '(1)',
  'Jay Wolpert',
  '(1)',
  'Jeff Nathanson',
  '(5)'],
 'Based on': ['Based on',
  'Pirates of the Caribbean',
  'by',
  'Walt Disney',
  'On Stranger Tides',
  'by',
  'Tim Powers',
  '(4)'],
 'Produced by': 'Jerry Bruckheimer',
 'Starring': ['Starring',
  'Johnny Depp',
  '(1–5)',
  'Geoffrey Rush',
  '(1–5)',
  'Kevin McNally',
  '(1–5)',
  'Orlando Bloom',
  '(1–3, 5)',
  'Keira Knightley',
  '(1–3, 5)',
  '(',
  'See below',
  ')'],
 'M

In [6]:
for movie_info in movie_info_list:
    print(movie_info.get('Budget'))

$1.5 million
$2.6 million
$2.28 million
$600,000
$950,000
$858,000
None
$788,000
None
$1.35 million
$2.125 million
None
$1.5 million
$1.5 million
None
$2.2 million
$1.8 million
$3 million
None
$4 million
$2 million
$300,000
$1.8 million
None
$5 million
None
$4 million
None
None
None
None
None
None
$700,000
None
None
None
None
None
$6 million
under $1 millionor $1,250,000
None
$2 million
None
None
$2.5 million
None
None
$4 million
$3.6 million
None
None
None
None
$3 million
None
$3 million
None
None
None
None
None
None
None
None
None
$3 million
None
None
None
None
$4.4–6 million
None
None
None
None
None
None
None
None
None
None
None
None
$4 million
None
$5 million
None
None
None
None
$5 million
None
None
None
None
None
None
$4 million
None
None
None
$6.3 million
None
None
None
None
None
None
None
None
$5 million
None
None
None
None
$8 million
None
None
None
None
None
None
AU$1 million
None
None
None
None
$5 million
None
None
$7.5 million
None
$10 million
None
None
$3.5 to 4 million
None

In [7]:
movie_info_list[-9]

{'title': 'The Rocketeer',
 'Release dates': [''],
 'Production companies': ['WaltDisneyPictures',
  'TouchstonePictures',
  'SilverScreenPartnersIV',
  'GordonCompany'],
 'Directed by': 'Joe Johnston',
 'Screenplay by': ['Danny Bilson', 'Paul De Meo'],
 'Story by': ['Danny Bilson', 'Paul De Meo', 'William Dear'],
 'Based on': ['Based on', 'The Rocketeer', 'by', 'Dave Stevens'],
 'Produced by': ['Charles Gordon', 'Lawrence Gordon', 'Lloyd Levin'],
 'Starring': ['Billy Campbell',
  'Alan Arkin',
  'Jennifer Connelly',
  'Paul Sorvino',
  'Timothy Dalton'],
 'Cinematography': 'Hiro Narita',
 'Edited by': ['Michael A. Stevenson', 'Arthur Schmidt'],
 'Music by': 'James Horner',
 'Distributed by': 'Buena Vista Pictures Distribution',
 'Release date': ['June 21, 1991(1991-06-21)'],
 'Running time': 108,
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$35–40 million',
 'Box office': '$46.7 million (U.S.)'}

import re

def take_lower(budget_data):
    budget_data = str(budget_data).replace('$', '').replace('Budget', '')
    if '-' in budget_data:
        all_values = budget_data.split('-', 2) #return value from data in range form
        return_lower = all_values[1]
    else:
        return budget_data

def clean_budget(budget_data):
    try:
        budget_data = take_lower(budget_data)

        if 'million' in budget_data:
            # Extract the numeric part of the budget string
            numeric_part = re.search(r'\d+\.?\d*', budget_data)
            if numeric_part:
                budget_in_million = float(numeric_part.group())  # Extracted numeric value
                return int(budget_in_million * 1000000)  # Convert to integer in millions
    except TypeError:
        return None

In [8]:
import re

def take_lower(budget_data):
    budget_data = str(budget_data).replace('$', '').replace('Budget', '')
    if '-' in budget_data:
        all_values = budget_data.split('-', 2)
        return_lower = all_values[1]
    else:
        return budget_data

def clean_budget(budget_data):
    try:
        budget_data = take_lower(budget_data)

        if 'million' in budget_data:
            # Extract the numeric part of the budget string
            numeric_part = re.search(r'\d+\.?\d*', budget_data)
            if numeric_part:
                budget_in_million = float(numeric_part.group())  # Extracted numeric value
                return int(budget_in_million * 1000000)  # Convert to integer in millions
    except TypeError:
        return None

# Define a function to return 0 if cleaned_budget is None
def get_budget_with_default(movie_info):
    budget_data = movie_info.get('Budget')  
    if budget_data is not None:  # Check if 'Budget' key exists
        cleaned_budget = clean_budget(budget_data)
        if cleaned_budget is None:
            return 0  # Return 0 when cleaned_budget is None
        else:
            return cleaned_budget
    else:
        return 0  # Return 0 when 'Budget' is not present

In [9]:
for movie_info in movie_info_list:
    budget_data = movie_info.get('Budget')  

    # Define a function to get the budget with a default value of 0
    def get_budget_with_default(budget_data):
        if budget_data is not None:
            cleaned_budget = clean_budget(budget_data)
            if cleaned_budget is None:
                return 0  # Return 0 when cleaned_budget is None
            else:
                return cleaned_budget
        else:
            return 0  # Return 0 when 'Budget' is not present

    # Use the function to get the cleaned budget
    cleaned_budget = get_budget_with_default(budget_data)

    # Update the 'Budget' key in 'movie_info' with the cleaned budget
    movie_info['Budget'] = cleaned_budget

    # Print the cleaned_budget value
    print(cleaned_budget)

1500000
2600000
2280000
0
0
0
0
0
0
1350000
2125000
0
1500000
1500000
0
2200000
1800000
3000000
0
4000000
2000000
0
1800000
0
5000000
0
4000000
0
0
0
0
0
0
0
0
0
0
0
0
6000000
1000000
0
2000000
0
0
2500000
0
0
4000000
3600000
0
0
0
0
3000000
0
3000000
0
0
0
0
0
0
0
0
0
3000000
0
0
0
0
4400000
0
0
0
0
0
0
0
0
0
0
0
0
4000000
0
5000000
0
0
0
0
5000000
0
0
0
0
0
0
4000000
0
0
0
6300000
0
0
0
0
0
0
0
0
5000000
0
0
0
0
8000000
0
0
0
0
0
0
1000000
0
0
0
0
5000000
0
0
7500000
0
10000000
0
0
3500000
0
0
0
5250000
20000000
4500000
9000000
0
0
20000000
0
0
18000000
12000000
14000000
10000000
17000000
5000000
8000000
20000000
11000000
28000000
44000000
0
7500000
14000000
9000000
0
8700000
31000000
18000000
5000000
40000000
20000000
0
14000000
60000000
0
35000000
25000000
15000000
32000000
14000000
28000000
12000000
0
0
11000000
28000000
17000000
30000000
0
13000000
0
0
5000000
45000000
31000000
0
22000000
30000000
0
0
22000000
32000000
18000000
55000000
24000000
15000000
12000000
0
30000000
0
310

In [10]:
movie_info_list[-9]

{'title': 'The Rocketeer',
 'Release dates': [''],
 'Production companies': ['WaltDisneyPictures',
  'TouchstonePictures',
  'SilverScreenPartnersIV',
  'GordonCompany'],
 'Directed by': 'Joe Johnston',
 'Screenplay by': ['Danny Bilson', 'Paul De Meo'],
 'Story by': ['Danny Bilson', 'Paul De Meo', 'William Dear'],
 'Based on': ['Based on', 'The Rocketeer', 'by', 'Dave Stevens'],
 'Produced by': ['Charles Gordon', 'Lawrence Gordon', 'Lloyd Levin'],
 'Starring': ['Billy Campbell',
  'Alan Arkin',
  'Jennifer Connelly',
  'Paul Sorvino',
  'Timothy Dalton'],
 'Cinematography': 'Hiro Narita',
 'Edited by': ['Michael A. Stevenson', 'Arthur Schmidt'],
 'Music by': 'James Horner',
 'Distributed by': 'Buena Vista Pictures Distribution',
 'Release date': ['June 21, 1991(1991-06-21)'],
 'Running time': 108,
 'Country': 'United States',
 'Language': 'English',
 'Budget': 35000000,
 'Box office': '$46.7 million (U.S.)'}

In [11]:
for movie_info in movie_info_list:
    print(movie_info.get('Box office'))

$418 million
$164 million
$76.4–$83.3 million (United States and Canada)
$960,000 (worldwide rentals)
>$1.3 million (est. United States/Canada rentals, 1941)
$267.4 million
$1 million (worldwide rentals)
$799,000
$3.355 million (worldwide rentals)
$3.275 million (worldwide rentals)
$65 million
$3.165 million (worldwide rentals)
$2.56 million (worldwide rentals)
$3.7 million (U.S. rental) + $575,000 (foreign rental)
$1.625 million (worldwide rentals)
$182 million
$4.1 million (worldwide rentals)
['$2.4 million (1951, domestic)', '$3.5 million (1974, domestic)']
$2.1 million (US rentals)
$87.4 million (United States and Canada)
$1 million (US)
$2.6 million (US)
None
$1.75 million (US and Canadian rentals)
$28.2 million
$2,150,000 (US)
$187 million
$2.1 million (US)
$1.6 million (US)
$1.7 million (US)
None
None
$2.75 million (US)
None
$1.75 million (US rentals)
$6.25 million (U.S./Canada rentals)
$2 million
$1.8 million (est. US/Canada rentals)
$2.5 million (est. US/ Canada rentals)
$51.6

In [12]:
import re

def remove_values_in_parentheses(bo_data):
    # Ensure bo_data is a string
    bo_data = str(bo_data)
    
    # Define a regex pattern to match values within parentheses
    pattern = r'\s*\([^)]*\)'

    # Use re.sub() to replace the matched patterns with an empty string
    cleaned_bo_data = re.sub(pattern, '', bo_data)

    return cleaned_bo_data






def clean_boxoffice(bo_data):
    # Remove values within parentheses
    bo_data = remove_values_in_parentheses(bo_data)

    # Convert to string and remove "$", "Box office", ">", and "₹"
    bo_data = str(bo_data).replace('$', '').replace('Box office', '').replace('>', '').replace('₹', '').replace(',','')

    # Check if the bo_data contains a hyphen or an em dash
    if '-' in bo_data or '–' in bo_data:
        # Use regular expressions to extract the upper limit of the box office range
        bo_match = re.search(r'(\d+(\.\d+)?)[–-](\d+(\.\d+)?)\s*million', bo_data)

        if bo_match:
            lower_limit_str = bo_match.group(1)
            upper_limit_str = bo_match.group(3)
            # Remove any non-digit characters (except for decimal point) and convert to float
            lower_limit_str = ''.join(filter(lambda x: x.isdigit() or x == '.', lower_limit_str))
            upper_limit_str = ''.join(filter(lambda x: x.isdigit() or x == '.', upper_limit_str))
            lower_limit = float(lower_limit_str) * 1_000_000
            upper_limit = float(upper_limit_str) * 1_000_000
            return int((lower_limit + upper_limit) / 2)
    elif 'million' in bo_data.lower():
        # Replace 'million' with '' and handle decimal points
        value = re.search(r'(\d+(\.\d+)?)\s*million', bo_data.lower()).group(1)
        try:
            value = float(value) * 1_000_000
            return int(value)
        except ValueError:
            pass
    elif 'crore' in bo_data.lower():
        # Change it to USD for uniformity, 1 crore = 10 million
        value = re.search(r'(\d+(\.\d+)?)\s*crore', bo_data.lower()).group(1)
        try:
            value = float(value) * 10_000_000 * 0.012
            return int(value)
        except ValueError:
            pass
    elif 'billion' in bo_data.lower():
        # Replace 'million' with '' and handle decimal points
        value = re.search(r'(\d+(\.\d+)?)\s*billion', bo_data.lower()).group(1)
        try:
            value = float(value) * 1_000_000_000
            return int(value)
        except ValueError:
            pass

    # If none of the conditions match, return 0 as a default
    return 0

In [13]:
for movie_info in movie_info_list:
  # Get the 'Box office' value, default to an empty string
  bo_data = movie_info.get('Box office', '')

  # Clean the 'Box office' value
  cleaned_boxoffice = clean_boxoffice(bo_data)

  # Check if the 'cleaned_boxoffice' variable is equal to 'None'
  if cleaned_boxoffice is None:
    # If it is, set the 'Box office' value in the 'movie_info' dictionary to 0
    movie_info['Box office'] = 0
  else:
    # Otherwise, try to convert the 'cleaned_boxoffice' variable to an integer
    try:
      movie_info['Box office'] = int(cleaned_boxoffice)
    except ValueError:
      # Handle the ValueError (invalid literal for int())
      movie_info['Box office'] = 0

  # Print the cleaned_boxoffice value
  print(int(cleaned_boxoffice)) 

418000000
164000000
79850000
0
1300000
267399999
1000000
0
3355000
3275000
65000000
3165000
2560000
3700000
1625000
182000000
4099999
2400000
2100000
87400000
1000000
2600000
0
1750000
28200000
0
187000000
2100000
1600000
1700000
0
0
2750000
0
1750000
6250000
2000000
1800000
2500000
51600000
12300000
0
0
0
0
3750000
2300000
0
40000000
303000000
25300000
25100000
2100000
1400000
4600000
3500000
5000000
0
0
0
0
22100000
0
0
0
4200000
22200000
0
0
0
0
103100000
0
0
0
0
0
6200000
0
0
0
0
0
0
378000000
0
5000000
0
0
0
3300000
51200000
1300000
0
5500000
0
0
4000000
191000000
0
0
5100000
17900000
0
0
0
0
0
0
0
2000000
33000000
7000000
38200000
4000000
0
10000000
6600000
0
0
5500000
0
0
0
0
0
0
10500000
36000000
0
0
169000000
28000000
37800000
0
0
0
0
0
0
4475000
35800000
2900000
5000000
18000000
11000000
60000000
16000000
0
14100000
63500000
0
8000000
50000000
7400000
0
8400000
27600000
11100000
21300000
9700000
13000000
38700000
18600000
22300000
0
121000000
222700000
0
235000000
18100000
47

In [14]:
movie_info_list[-11]

{'title': 'Three Men and a Baby',
 'Release dates': [''],
 'Production companies': ['TouchstonePictures',
  'SilverScreenPartnersIII',
  'InterscopeCommunications'],
 'Directed by': 'Leonard Nimoy',
 'Screenplay by': ['Jim Cruickshank', 'James Orr'],
 'Based on': ['Based on',
  'Trois hommes et un couffin',
  'by',
  'Coline Serreau'],
 'Produced by': ['Ted Field', 'Robert W. Cort'],
 'Starring': ['Tom Selleck',
  'Steve Guttenberg',
  'Ted Danson',
  'Nancy Travis',
  'Margaret Colin',
  'Celeste Holm'],
 'Cinematography': 'Adam Greenberg',
 'Edited by': 'Michael A. Stevenson',
 'Music by': 'Marvin Hamlisch',
 'Distributed by': 'Buena Vista Pictures Distribution',
 'Release date': ['November 25, 1987(1987-11-25)'],
 'Running time': 102,
 'Country': 'United States',
 'Language': 'English',
 'Budget': 11000000,
 'Box office': 240000000}

In [15]:
movie_info_list[-9]

{'title': 'The Rocketeer',
 'Release dates': [''],
 'Production companies': ['WaltDisneyPictures',
  'TouchstonePictures',
  'SilverScreenPartnersIV',
  'GordonCompany'],
 'Directed by': 'Joe Johnston',
 'Screenplay by': ['Danny Bilson', 'Paul De Meo'],
 'Story by': ['Danny Bilson', 'Paul De Meo', 'William Dear'],
 'Based on': ['Based on', 'The Rocketeer', 'by', 'Dave Stevens'],
 'Produced by': ['Charles Gordon', 'Lawrence Gordon', 'Lloyd Levin'],
 'Starring': ['Billy Campbell',
  'Alan Arkin',
  'Jennifer Connelly',
  'Paul Sorvino',
  'Timothy Dalton'],
 'Cinematography': 'Hiro Narita',
 'Edited by': ['Michael A. Stevenson', 'Arthur Schmidt'],
 'Music by': 'James Horner',
 'Distributed by': 'Buena Vista Pictures Distribution',
 'Release date': ['June 21, 1991(1991-06-21)'],
 'Running time': 108,
 'Country': 'United States',
 'Language': 'English',
 'Budget': 35000000,
 'Box office': 46700000}

In [16]:
for movie_info in movie_info_list:
    print(movie_info.get('Release date'))

None
None
['November 13, 1940(1940-11-13)']
['June 27, 1941(1941-06-27)']
None
None
None
['July 17, 1943(1943-07-17)']
None
None
None
['September 27, 1947(1947-09-27)']
['M']
None
['October 5, 1949(1949-10-05)']
None
None
None
None
['February 5, 1953(1953-02-05)']
['July 23, 1953(1953-07-23)(United States)']
['November 10, 1953(1953-11-10)']
None
['August 17, 1954(1954-08-17)']
['December 23, 1954(1954-12-23)']
['M']
['June 22, 1955(1955-06-22)']
['September 14, 1955(1955-09-14)']
['D']
['J']
['July 18, 1956(1956-07-18)']
['September 4, 1956(1956-09-04)']
['December 20, 1956(1956-12-20)']
['J']
['A']
['December 25, 1957(1957-12-25)']
['July 8, 1958(1958-07-08)']
['August 12, 1958(1958-08-12)']
['December 25, 1958(1958-12-25)']
['January 29, 1959(1959-01-29)']
['March 19, 1959(1959-03-19)']
None
['November 10, 1959(1959-11-10)']
['J']
['February 24, 1960(1960-02-24)']
['M']
None
['November 1, 1960(1960-11-01)']
['December 21, 1960(1960-12-21)']
['January 25, 1961(1961-01-25)']
['M']
['J

In [17]:
# Update each dictionary in movie_info_list with the combined release date
for movie_info in movie_info_list:
    movie_info['The release date'] = movie_info.get('Release date') if movie_info.get('Release date') is not None else movie_info.get('Release dates')

# Print the updated movie_info_list
for movie_info in movie_info_list:
    print(movie_info.get('The release date'))

['December 21, 1937(1937-12-21)(Carthay Circle Theatre)']
['February 7, 1940(1940-02-07)(Center Theatre)']
['November 13, 1940(1940-11-13)']
['June 27, 1941(1941-06-27)']
['October 23, 1941(1941-10-23)(New York City)']
['August 9, 1942(1942-08-09)(London)']
['August 24, 1942(1942-08-24)(Rio de Janeiro)']
['July 17, 1943(1943-07-17)']
['December 21, 1944(1944-12-21)(Mexico City)']
['April 20, 1946(1946-04-20)(New York City)']
['November 12, 1946(1946-11-12)(premiere)']
['September 27, 1947(1947-09-27)']
['M']
['November 29, 1948 (Chicago)']
['October 5, 1949(1949-10-05)']
['February 15, 1950(1950-02-15)(Boston)']
['June 22, 1950(1950-06-22)(London)']
['July 26, 1951(1951-07-26)(London)']
['13 March 1952(1952-03-13)(London)']
['February 5, 1953(1953-02-05)']
['July 23, 1953(1953-07-23)(United States)']
['November 10, 1953(1953-11-10)']
['26 October 1953(1953-10-26)(London)']
['August 17, 1954(1954-08-17)']
['December 23, 1954(1954-12-23)']
['M']
['June 22, 1955(1955-06-22)']
['September 

In [18]:
import re
import pendulum

def clean_date(date_data):
    # Ensure date_data is a string
    date_data = str(date_data)
    
    # Define a regular expression pattern to extract the numerical date
    pattern = r'\(\d{4}-\d{2}-\d{2}\)'  # Matches (YYYY-MM-DD)
    
    # Extract all numerical dates from the string
    numerical_dates = re.findall(pattern, date_data)

    if numerical_dates:
        # Take the first numerical date (if there are multiple) and remove parentheses
        numerical_date = numerical_dates[0].strip('()')

        # Convert the extracted date string to a pendulum datetime object
        try:
            pendulum_date = pendulum.parse(numerical_date)
            
            # Extract the date as a datetime.date object
            date_obj = pendulum_date.date()
            
            return date_obj
        except pendulum.parsing.exceptions.ParserError:
            return None  # Return None if the date couldn't be parsed
    else:
        return None  # Return None if no valid date pattern is found

In [19]:
for movie_info in movie_info_list:
    date_data = movie_info.get('The release date', '')
    cleaned_date = clean_date(date_data)
    movie_info['The release date'] = cleaned_date
    print(cleaned_date) 

1937-12-21
1940-02-07
1940-11-13
1941-06-27
1941-10-23
1942-08-09
1942-08-24
1943-07-17
1944-12-21
1946-04-20
1946-11-12
1947-09-27
None
None
1949-10-05
1950-02-15
1950-06-22
1951-07-26
1952-03-13
1953-02-05
1953-07-23
1953-11-10
1953-10-26
1954-08-17
1954-12-23
None
1955-06-22
1955-09-14
None
None
1956-07-18
1956-09-04
1956-12-20
None
None
1957-12-25
1958-07-08
1958-08-12
1958-12-25
1959-01-29
1959-03-19
1959-06-24
1959-11-10
None
1960-02-24
None
None
1960-11-01
1960-12-21
1961-01-25
None
1961-06-21
1961-07-12
1961-07-17
1961-12-14
None
1962-05-17
1962-06-06
None
1962-11-07
1962-11-14
1963-01-16
1963-03-29
1963-06-01
1963-07-07
None
1963-12-12
1964-03-12
1964-02-11
1963-12-11
1964-07-02
1964-08-27
None
None
1964-12-18
1965-08-18
1965-12-02
1966-02-04
1966-06-29
None
1966-12-01
1967-02-08
1967-06-15
1967-07-12
1967-10-18
1967-10-18
1967-06-23
1968-02-08
1968-03-21
1968-06-26
1968-12-20
1968-12-24
1969-03-21
1969-06-11
1969-12-31
1970-02-11
None
1970-12-15
1970-12-11
1971-03-17
None
197

In [24]:
release_date_value = movie_info['The release date']
data_type = type(release_date_value)
print(data_type)

<class 'pendulum.date.Date'>


In [20]:
movie_info_list[-11]

{'title': 'Three Men and a Baby',
 'Release dates': [''],
 'Production companies': ['TouchstonePictures',
  'SilverScreenPartnersIII',
  'InterscopeCommunications'],
 'Directed by': 'Leonard Nimoy',
 'Screenplay by': ['Jim Cruickshank', 'James Orr'],
 'Based on': ['Based on',
  'Trois hommes et un couffin',
  'by',
  'Coline Serreau'],
 'Produced by': ['Ted Field', 'Robert W. Cort'],
 'Starring': ['Tom Selleck',
  'Steve Guttenberg',
  'Ted Danson',
  'Nancy Travis',
  'Margaret Colin',
  'Celeste Holm'],
 'Cinematography': 'Adam Greenberg',
 'Edited by': 'Michael A. Stevenson',
 'Music by': 'Marvin Hamlisch',
 'Distributed by': 'Buena Vista Pictures Distribution',
 'Release date': ['November 25, 1987(1987-11-25)'],
 'Running time': 102,
 'Country': 'United States',
 'Language': 'English',
 'Budget': 11000000,
 'Box office': 240000000,
 'The release date': Date(1987, 11, 25)}

In [21]:
# Get the column headers for all the dictionaries in the list
column_headers = []
for movie_info in movie_info_list:
    column_headers.extend(list(movie_info.keys()))

# Convert the list of column headers into a set to remove duplicates
distinct_column_headers = set(column_headers)

# Print the distinct column headers
print(distinct_column_headers)

{'Production company', 'Release date', 'Written by', 'Lyrics', 'Cinematography', 'Color process', 'Awarded for', 'Website', 'Narrated by', 'Presented by', 'Story by', 'Budget', 'Country', 'Productions', 'Awards', 'Currently held by', 'Layouts by', 'title', 'Screenplay by', 'Language', 'Countries', 'Based on', 'Music', 'Directed by', 'Box office', 'Edited by', 'Basis', 'The release date', 'Languages', 'Starring', 'Location', 'Release dates', 'Production companies', 'Produced by', 'Music by', 'Distributed by', 'Running time', 'Book'}


In [30]:
import pendulum

# Initialize a dictionary to store the data types for each column
column_data_types = {}

# Iterate through the dictionaries to determine data types
for movie_info in movie_info_list:
    for column_name, column_value in movie_info.items():
        if column_name not in column_data_types:
            # If the column name is not in the dictionary, add it with the data type of the first non-empty value encountered
            if column_value != '':
                if isinstance(column_value, pendulum.Date):
                    column_data_types[column_name] = pendulum.Date
                else:
                    column_data_types[column_name] = type(column_value)
        else:
            # If the column name is already in the dictionary, update the data type if a different type is encountered
            current_data_type = column_data_types[column_name]
            if column_value != '' and type(column_value) != current_data_type:
                # Handle cases where the data type is different
                if isinstance(column_value, pendulum.Date):
                    column_data_types[column_name] = pendulum.Date
                else:
                    column_data_types[column_name] = str

# Print column names and their corresponding data types
for column_name, data_type in column_data_types.items():
    print(f"Column: {column_name}, Data Type: {data_type.__name__}")


Column: title, Data Type: str
Column: Release dates, Data Type: list
Column: Production companies, Data Type: list
Column: Directed by, Data Type: str
Column: Story by, Data Type: str
Column: Based on, Data Type: str
Column: Produced by, Data Type: str
Column: Starring, Data Type: str
Column: Music by, Data Type: str
Column: Production company, Data Type: list
Column: Distributed by, Data Type: str
Column: Running time, Data Type: int
Column: Country, Data Type: str
Column: Language, Data Type: str
Column: Budget, Data Type: int
Column: Box office, Data Type: int
Column: The release date, Data Type: Date
Column: Narrated by, Data Type: str
Column: Cinematography, Data Type: str
Column: Release date, Data Type: list
Column: Written by, Data Type: str
Column: Edited by, Data Type: str
Column: Languages, Data Type: str
Column: Screenplay by, Data Type: str
Column: Countries, Data Type: list
Column: Color process, Data Type: str
Column: Awarded for, Data Type: str
Column: Location, Data Ty